In [34]:
!pip install pandas xlrd openpyxl

                                              0.0/250.0 kB ? eta -:--:--
     --------------                          92.2/250.0 kB 1.7 MB/s eta 0:00:01
     -------------------------------        204.8/250.0 kB 2.1 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 1.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Завдання №1
# Завантаження даних з файлу Excel і виключення верхніх та нижніх рядків
energy = pd.read_excel('En_In.xls', skiprows=17, skipfooter=38)
# Видалення перших двох стовпців
energy = energy.iloc[:, 2:]
# Перейменування стовпців
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

# Завдання №2
# Заміна значень '...' на NaN
energy = energy.replace('...', np.nan)
# Переведення 'Energy Supply' у гігаджоулі
energy['Energy Supply'] = energy['Energy Supply'] * 1000000

# Завдання №3
# Словник для перейменування країн
country_rename_dict = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
# Заміна назв країн за вказаним словником
energy['Country'].replace(country_rename_dict, inplace=True)

# Завдання №4
# Функція для видалення цифр та дужок з назв країн
def remove_numbers_and_brackets(name):
    return re.sub(r'\s*\(.*?\)|\d+', '', name).strip()
# Застосування функції до стовпця 'Country'
energy['Country'] = energy['Country'].apply(remove_numbers_and_brackets)

energy.loc[energy['Country'].isin(['American Samoa' ,'South Korea', 'Bolivia'])]

,Country,Energy Supply,Energy Supply per Capita,% Renewable
3,American Samoa,NaN,NaN,0.641026
24,Bolivia,3.360000e+08,32.0,31.477120
164,South Korea,1.100700e+10,221.0,2.279353


In [32]:
# Завдання №5
# Завантаження даних про ВВП
gdp = pd.read_csv('gpd.csv', skiprows=4)

# Перейменування країн
country_rename_dict_gdp = {
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}
gdp['Country Name'].replace(country_rename_dict_gdp, inplace=True)

# Вивід перших 11 стовпців
gdp.head(1)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN


In [71]:
# Завантаження даних з файлу "scimagojr.xlsx"
scimagojr = pd.read_excel('scimagojr.xlsx')

# Об'єднання даних за назвою країни
merged = pd.merge(energy, gdp, how='inner', left_on='Country', right_on='Country Name')
merged = pd.merge(merged, scimagojr[scimagojr['Rank'] <= 15], how='inner', left_on='Country', right_on='Country')

# Обрання необхідних стовпців та даних за останні 10 років
years = [str(year) for year in range(2006, 2016)]
columns = ['Country', 'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
           'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] + years
final_data = merged[columns]

# Встановлення назв країн як індексу DataFrame
final_data.set_index('Country', inplace=True)

final_data.head(3)

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14,8831,8725,90765,15606,10.28,107,5.386000e+09,231.0,11.81081,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.64803,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.94543,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12


In [51]:
final_data.shape

(12, 10)